In [1]:
import os
exec(open("Libs_Modulus.py").read())
exec(open("Links.py").read())
exec(open("ConvertTool.py").read())
"""exec(open("CheckDirFiles.py").read())"""

def CHECK_DATA_DIR(MainDir,DataType):
    if not os.path.exists(MainDir+DataType):
        os.mkdir(MainDir+DataType)
        ExistDir = False
    else :
        ExistDir = True
    print("Le répertoire '", DataType, "' exist : ", np.where(ExistDir==True, "OK", "NOK \nCréation du répertoire"))
    return ExistDir

# Mise à jour des fichiers Excel téléchargés pour les rendre exploitables
def UPDATE_FORMAT(file_name, name_filter, change):
    tempdir = tempfile.mkdtemp()
    try:
        tempname = os.path.join(tempdir, 'new.zip')
        with ZipFile(file_name, 'r') as r, ZipFile(tempname, 'w') as w:
            for item in r.infolist():
                data = r.read(item.filename)           
                data = change(data)
                w.writestr(item, data)
        shutil.move(tempname, file_name)
    finally:
        shutil.rmtree(tempdir)

# Obtention de la liste des fichiers de classements journaliers
def GET_RANK_LIST (RankURL):
    soup = BeautifulSoup(requests.get(RankURL).content.decode("utf-8"))
    ListSoup = soup.findAll('option')
    RankList = []
    for iRankList in ListSoup:
        RankList.append(iRankList.text[2:])
    RankList = np.unique(RankList[1:-1])
    print ("Il y a ", len(RankList), "fichiers Excel de classement")
    return RankList
        
# Téléchargement de fichier de classement journalier
def DWNLD_DAILY_RANK_FILE(DLRankURL, TFN, RankFileType, MainDir, DataDir):
    os.chdir(MainDir+DataDir)
    
    with tqdm(range(len(TFN)), desc = "Chargement des données Excel") as outer:
        for iTFN in TFN:
            resp = requests.get(DLRankURL+iTFN+RankFileType)
            print("vendeeglobe_"+iTFN+RankFileType)
            with open("vendeeglobe_"+iTFN+RankFileType,'wb') as file:
                file.write(resp.content)
            UPDATE_FORMAT("vendeeglobe_"+iTFN+RankFileType, name_filter='xl/styles.xml', change=lambda d: re.sub(b'xxid="\d*"', b"", d))
            outer.update()
    os.chdir(MainDir)

# Vérification de l'existance de tous les fichiers à télécharger
def CHECK_FILE_LIST(MainDir, DataRankDir, TFN):
    Directory = MainDir+DataRankDir
    FilesList = [f for f in listdir(Directory) if isfile(join(Directory, f))]
    #print(FilesList)
    ListToDownLoad = []
    for iTFN in TFN:
        if  not str("vendeeglobe_"+iTFN+".xlsx") in FilesList:
            ListToDownLoad.append(iTFN)
            pass
    return ListToDownLoad


""" MAIN """
ExistDir = CHECK_DATA_DIR(MainDir,DataRankDir)
RankList = GET_RANK_LIST (RankURL)
TL,TFN = CONVERT_TIME_FORMAT(RankList)
if ExistDir == False :
    DWNLD_DAILY_RANK_FILE(DLRankURL, TFN, RankFileType, MainDir, DataRankDir)
else :
    ListToDownLoad = CHECK_FILE_LIST(MainDir, DataRankDir, TFN)
    if len(ListToDownLoad)!=0:
        DWNLD_DAILY_RANK_FILE(DLRankURL, ListToDownLoad, RankFileType, MainDir, DataRankDir)


Le répertoire ' \DataRank_2021 ' exist :  OK
Il y a  702 fichiers Excel de classement


In [2]:
def GEN_SEPCS_DF (SkippersURL):
    # Paramètres des types de données à extraire
    DataParam = {"BoatSpecs" : ['ul', "class", "boats-list__popup-specs-list"],
                 "Nom voilier" : ['h3',"class","boats-list__boat-name"],
                 "Nom skipper" : ["span","class", "boats-list__skipper-name"]}
    # Fonction d'extraction de données techniques spécifiques (BoatSpecs, BoatName, SkipperName) sous forme de list
    LIST_SOUP = lambda SkippersURL, DataParam,ikey : BeautifulSoup(requests.get(SkippersURL).content.decode("utf-8"))\
    .findAll(DataParam[ikey][0],
             {DataParam[ikey][1]:
              DataParam[ikey][2]})
    
    # Création d'un DF Spécifications techniques du voilier vierge
    SpecsDF=pd.DataFrame()
    
    # Remplissage du DF
    for ikey in DataParam:
        ListSoup = LIST_SOUP (SkippersURL, DataParam, ikey)
        
        # Remplir d'abord les données techniques du voilier (Num. voile, Anciens nom, Architece, Chantier, Date lanc., Longueur,...)
        if ikey == list(DataParam.keys())[0]:
            i = 0
            for iul in ListSoup:
                idata,ivalue = np.transpose([string.split(' : ',1) for string in iul.text.split("\n")[1:-1]])
                SpecsDF = pd.concat([SpecsDF,pd.DataFrame([ivalue],columns = idata)],ignore_index=True)
                i+=1
        # Ajouter ensuite les colonnes Nom de voilier, et nom du marin; puis les mettre en premières colonnes
        else:
            SpecsDF[ikey] = [string.text for string in ListSoup]
            SpecsDF = SpecsDF[SpecsDF.columns.tolist()[-1:]+SpecsDF.columns.tolist()[:-1]]
    
    # Homogénéisation des données
       # liste des valeurs [init, new] et des unités à supprimer suivant le nom de la colonne 
    ValToReplace = {"Date de lancement":[], # pour reconvertion des dates
                    "Anciens noms du bateau":[np.nan,''],"Voile quille":[np.nan,''], # Pour remplacement des valeurs
                    "Nombre de dérives":[["foiler", "2 asymétriques"],['foils', "2"]], # Pour remplacement des valeurs
                    "Longueur":[' m'],"Largeur" :[' m'],"Tirant d'eau":[' m'],"Hauteur mât":[' m'], # Suppresion des unités de longueur
                    "Surface de voiles au près":[' m2', ' m²'],"Surface de voiles au portant":[' m2', ' m²'], #Suppression des unités de surface
                    "Déplacement (poids)":[' t', ' tonnes']} #Suppresion des unités de masse
    for icol in ValToReplace:
        if icol in ["Anciens noms du bateau","Voile quille","Nombre de dérives"]:      
            SpecsDF[icol] = SpecsDF[icol].replace(ValToReplace[icol][0],ValToReplace[icol][1])
        elif "Date de lancement" in icol:
            SpecsDF[icol] = SpecsDF[icol].apply(dateparser.parse)
        else :
            for iunit in ValToReplace[icol]:
                SpecsDF[icol] = SpecsDF[icol].str.strip(iunit)
            if "Déplacement (poids)" in icol:
                SpecsDF[icol] = SpecsDF[icol].str.replace('[a-zA-Z]', '0', regex=True)
            SpecsDF[icol] = SpecsDF[icol].str.replace(',','.')
            SpecsDF[icol] = SpecsDF[icol].astype('float')
    return SpecsDF

In [3]:
Specs_DF = GEN_SEPCS_DF(SkippersURL)
Specs_DF.head()

,Nom skipper,Nom voilier,Numéro de voile,Anciens noms du bateau,Architecte,Chantier,Date de lancement,Longueur,Largeur,Tirant d'eau,Déplacement (poids),Nombre de dérives,Hauteur mât,Voile quille,Surface de voiles au près,Surface de voiles au portant
0,Fabrice AMEDEO,NEWREST - ART & FENÊTRES,FRA 56,"No Way Back, Vento di Sardegna",VPLP/Verdier,Persico Marine,2015-08-01,18.28,5.85,4.5,7.0,foils,29.0,monotype,320.0,570.0
1,Romain ATTANASIO,PURE - Best Western®,FRA 49,"Gitana Eighty, Synerciel, Newrest-Matmut",Bruce Farr Design,Southern Ocean Marine (Nouvelle Zélande),2007-03-08,18.28,5.80,4.5,9.0,2,28.0,acier forgé,80.0,560.0
2,Alexia BARRIER,TSE - 4MYPLANET,FRA72,"Famille Mary-Etamine du Lys, Initiatives Coeur...",Marc Lombard,MAG France,1998-03-01,18.28,5.54,4.5,9.0,2,29.0,acier,60.0,580.0
3,Yannick BESTAVEN,Maître CoQ IV,17,Safran 2 - Des Voiles et Vous,Verdier - VPLP,CDK Technologies,2015-03-12,18.28,5.80,4.5,8.0,foils,29.0,acier mécano soudé,310.0,550.0
4,Jérémie BEYOU,CHARAL,08,,VPLP,CDK Technologies,2018-08-18,18.28,5.85,4.5,8.0,foils,29.0,acier,320.0,600.0


In [5]:
os.chdir(MainDir+DataRankDir)
df = pd.read_excel ("vendeeglobe_20210305_080000.xlsx", header=3)
df = df.rename(columns={'Unnamed: 0': 'Date'})
df["Date"]='2021/03/05 08:00:00'
print('Attention! Date en format str à convertir en format date')

df["Date d'arrivée\nArrival date"] = df["Unnamed: 7"]
df.loc[1:,"Écarts\nGaps"] = df.loc[1:,"Unnamed: 13"]
df.loc[1:,"Unnamed: 14"] = df.loc[1:,"Unnamed: 15"]
df = df.drop(["Unnamed: 5","Unnamed: 6","Unnamed: 7", "Unnamed: 9", "Unnamed: 10", "Unnamed: 11", "Unnamed: 13", "Unnamed: 15"], 1)
df.head()

Attention! Date en format str à convertir en format date


C:\Users\alexa\AppData\Local\Temp/ipykernel_10580/3336559639.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop(["Unnamed: 5","Unnamed: 6","Unnamed: 7", "Unnamed: 9", "Unnamed: 10", "Unnamed: 11", "Unnamed: 13", "Unnamed: 15"], 1)


,Date,Rang\nRank,Nat. / Voile\nNat. / Sail,Skipper / Bateau\nSkipper / crew,Date d'arrivée\nArrival date,Temps de course\nRace time,Écarts\nGaps,Unnamed: 14,Sur l'ortho\nOver ortho,Unnamed: 17,►,Sur le fond\nOver ground,Unnamed: 20
0,2021/03/05 08:00:00,NaN,NaN,NaN,NaN,NaN,Au premier\nTo first,Au précédent\nTo previous,Vitesse\nSpeed,Distance\nDistance,NaN,Vitesse\nSpeed,Distance\nDistance
1,2021/03/05 08:00:00,1\nARV,\nFRA 17,Yannick Bestaven\nMaître Coq IV,28/01/2021 04:19:46 FR,80j 03h 44min 46s\n-10h 15min 00s,NaN,NaN,12.6 kts,24365.7 nm,117.3 %,14.8 kts,28583.8 nm
2,2021/03/05 08:00:00,2\nARV,\nFRA 79,Charlie Dalin\nAPIVIA,27/01/2021 20:35:47 FR,80j 06h 15min 47s\n,02h 31min 01s,02h 31min 01s,12.6 kts,24365.7 nm,119.6 %,15.1 kts,29135.0 nm
3,2021/03/05 08:00:00,3\nARV,\nFRA 18,Louis Burton\nBureau Vallée 2,28/01/2021 00:45:12 FR,80j 10h 25min 12s\n,06h 40min 26s,04h 09min 25s,12.6 kts,24365.7 nm,117.6 %,14.8 kts,28650.0 nm
4,2021/03/05 08:00:00,4\nARV,\nFRA 01,Jean Le Cam\nYes we Cam !,28/01/2021 20:19:55 FR,80j 13h 44min 55s\n-16h 15min 00s,10h 00min 09s,03h 19min 43s,12.5 kts,24365.7 nm,112.9 %,14.1 kts,27501.5 nm


In [31]:
os.chdir(MainDir+DataRankDir)
print(MainDir+DataRankDir)
df = pd.read_excel ("vendeeglobe_20201108_080000.xlsx", header=3)

D:\TELECOM PARIS\GitHub\Kit_BgD_AlexandreROULEAU\211103_FinalProject\DataRank_2021


FileNotFoundError: [Errno 2] No such file or directory: 'vendeeglobe_20201108_080000.xlsx'

In [30]:
os.chdir(MainDir+DataRankDir)
print(MainDir+DataRankDir)
df = pd.read_excel ("vendeeglobe_20201109_080000.xlsx", header=3)

D:\TELECOM PARIS\GitHub\Kit_BgD_AlexandreROULEAU\211103_FinalProject\DataRank_2021


In [46]:
TFN[4]

'20201109_040000'

In [53]:
pd.read_excel ("vendeeglobe_"+TFN[0]+".xlsx", header=3)

ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [52]:
for i in TFN[5:]:
    print (i)
    df = pd.read_excel ("vendeeglobe_"+i+".xlsx", header=3)

20201109_080000
20201109_110000
20201109_140000
20201109_170000
20201109_210000
20201110_040000
20201110_080000
20201110_110000
20201110_140000
20201110_170000
20201110_210000
20201111_040000
20201111_080000
20201111_110000
20201111_140000
20201111_170000
20201111_210000
20201112_040000
20201112_080000
20201112_110000
20201112_140000
20201112_170000
20201112_210000
20201113_040000
20201113_080000
20201113_110000
20201113_140000
20201113_170000
20201113_210000
20201114_040000
20201114_080000
20201114_110000
20201114_140000
20201114_170000
20201114_210000
20201115_040000
20201115_080000
20201115_110000
20201115_140000
20201115_170000
20201115_210000
20201116_040000
20201116_080000
20201116_110000
20201116_140000
20201116_170000
20201116_210000
20201117_040000
20201117_080000
20201117_110000
20201117_140000
20201117_170000
20201117_210000
20201118_040000
20201118_080000
20201118_110000
20201118_140000
20201118_170000
20201118_210000
20201119_040000
20201119_080000
20201119_110000
20201119

In [44]:
os.chdir(MainDir+DataRankDir)
for i in RankList[4:]:
    df = pd.read_excel ("vendeeglobe_20201109_080000.xlsx", header=3)

KeyboardInterrupt: 

In [17]:
os.chdir(MainDir+DataRankDir)
print(MainDir+DataRankDir)
df = pd.read_excel ("vendeeglobe_20201201_040000.xlsx", header=3)
"""df = df.rename(columns={'Unnamed: 0': 'Date'})
df["Date"]='2021/03/05 14:00:00'
print('Attention! Date en format str à convertir en format date')

df["Date d'arrivée\nArrival date"] = df["Unnamed: 7"]
df.loc[1:,"Écarts\nGaps"] = df.loc[1:,"Unnamed: 13"]
df.loc[1:,"Unnamed: 14"] = df.loc[1:,"Unnamed: 15"]
df = df.drop(["Unnamed: 5","Unnamed: 6","Unnamed: 7", "Unnamed: 9", "Unnamed: 10", "Unnamed: 11", "Unnamed: 13", "Unnamed: 15"], 1)
df.head()"""
df

D:\TELECOM PARIS\GitHub\Kit_BgD_AlexandreROULEAU\211103_FinalProject\DataRank_2021


,Unnamed: 0,Rang\nRank,Nat. / Voile\nNat. / Sail,Skipper / Bateau\nSkipper / crew,Unnamed: 4,Unnamed: 5,Unnamed: 6,Depuis 30 minutes\nSince 30 minutes,Unnamed: 8,Unnamed: 9,...,Depuis le dernier classement\nSince the the last report,Unnamed: 12,Unnamed: 13,Unnamed: 14,Depuis 24 heures\nSince 24 hours,Unnamed: 16,Unnamed: 17,Unnamed: 18,DTF,DTL
0,NaN,NaN,NaN,NaN,Heure FR\nHour FR,Latitude\nLatitude,Longitude\nLongitude,Cap\nHeading,Vitesse\nSpeed,VMG\nVMG,...,Cap\nHeading,Vitesse\nSpeed,VMG\nVMG,Distance\nDistance,Cap\nHeading,Vitesse\nSpeed,VMG\nVMG,Distance\nDistance,NaN,NaN
1,NaN,1,\nFRA 79,Charlie Dalin\nAPIVIA,04:30 FR\n,41°41.41'S,19°46.18'E,102°,14.8 kts,14.8 kts,...,107°,14.0 kts,14.0 kts,97.9 nm,119°,16.3 kts,16.1 kts,391.9 nm,17487.2 nm,0.0 nm
2,NaN,2,\nFRA 59,Thomas Ruyant\nLinkedOut,04:30 FR\n,41°04.71'S,14°57.06'E,77°,15.8 kts,14.1 kts,...,87°,16.6 kts,15.9 kts,141.4 nm,97°,16.9 kts,16.6 kts,405.8 nm,17707.0 nm,219.8 nm
3,NaN,3,\nFRA 18,Louis Burton\nBureau Vallée 2,04:30 FR\n,44°25.15'S,12°43.31'E,93°,16.4 kts,16.3 kts,...,95°,17.2 kts,17.1 kts,120.1 nm,98°,18.1 kts,18.0 kts,434.1 nm,17766.9 nm,279.7 nm
4,NaN,4,\nFRA 1000,Damien Seguin\nGroupe APICIL,04:30 FR\n,41°54.88'S,09°58.25'E,70°,15.2 kts,12.6 kts,...,85°,17.7 kts,16.7 kts,123.7 nm,95°,17.1 kts,16.7 kts,411.0 nm,17912.5 nm,425.3 nm
5,NaN,5,\nMON 10,Boris Herrmann\nSeaexplorer - Yacht Club De Mo...,04:30 FR\n,40°47.00'S,09°56.50'E,97°,13.8 kts,13.7 kts,...,82°,6.6 kts,6.0 kts,45.9 nm,101°,13.4 kts,13.3 kts,322.4 nm,17931.0 nm,443.8 nm
6,NaN,6,\nFRA 4,Sébastien Simon\nARKEA PAPREC,04:30 FR\n,40°49.65'S,09°54.89'E,101°,21.3 kts,21.3 kts,...,26°,6.3 kts,1.2 kts,44.0 nm,100°,12.0 kts,11.8 kts,287.6 nm,17931.5 nm,444.3 nm
7,NaN,7,\nFRA 17,Yannick Bestaven\nMaître Coq IV,04:30 FR\n,40°36.56'S,09°50.68'E,62°,12.9 kts,9.2 kts,...,55°,4.4 kts,2.8 kts,30.5 nm,99°,10.4 kts,10.3 kts,249.4 nm,17938.2 nm,451.0 nm
8,NaN,8,\nFRA 01,Jean Le Cam\nYes we Cam !,04:30 FR\n,40°37.86'S,09°48.85'E,77°,16.4 kts,14.3 kts,...,68°,4.2 kts,3.3 kts,29.1 nm,104°,9.2 kts,9.2 kts,221.9 nm,17939.2 nm,452.0 nm
9,NaN,9,\nFRA 85,Kevin Escoffier\nPRB,15:00 FR\n-810min,40°56.67'S,09°16.33'E,115°,9.7 kts,9.6 kts,...,0°,0.0 kts,0.0 kts,0.0 nm,102°,18.0 kts,17.8 kts,432.6 nm,17957.7 nm,470.5 nm


In [ ]:
RankList

In [ ]:
from urllib import request
import shutil
url = "https://www.vendeeglobe.org/download-race-data/vendeeglobe_20201108_140000.xlsx"
filename = "vendeeglobe_20201108_140000.xlsx"

with request.urlopen(url) as response, open(filename, 'wb') as out_file: shutil.copyfileobj(response, out_file)

In [ ]:
df = pd.read_excel ("vendeeglobe_20201108_140000.xlsx",sheet_name=0, header=3)
df